# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import csv
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = '../output_data/cities.csv'
city_df = pd.read_csv(city_data)
city_df = city_df.set_index('City_ID')
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City_ID,,,,,,,,,
0,provideniya,90,RU,1596061967,87,64.38,-173.30,50.00,8.95
1,kurilsk,46,RU,1596062097,81,45.23,147.88,61.52,6.76
2,mariinsk,82,RU,1596062097,94,56.21,87.75,56.10,3.91
3,punta arenas,99,CL,1596061931,80,-53.15,-70.92,39.20,13.87
4,castro,100,BR,1596062097,97,-24.79,-50.01,53.74,7.70


In [3]:
city_df.dtypes

City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = city_df[["Lat", "Lng"]].astype(float)
humidity = city_df['Humidity'].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_df = city_df[(city_df['Max Temp']>70)&(city_df['Max Temp']<80)&(city_df['Wind Speed']<5)&(city_df['Cloudiness']==0)]
ideal_df = ideal_df.dropna()
ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City_ID,,,,,,,,,
14,lodja,0,CD,1596062099,42,-3.48,23.43,73.13,0.43
87,burriana,0,ES,1596062151,84,39.89,-0.09,78.01,3.53
88,barcelos,0,PT,1596061919,93,41.54,-8.62,73.00,1.01
111,lata,0,PT,1596062156,69,40.16,-8.33,73.00,2.68
131,komsomolskiy,0,UZ,1596061980,44,40.43,71.72,75.20,4.70
149,arles,0,FR,1596062165,65,43.68,4.63,78.80,2.24
157,izvestkovyy,0,RU,1596062166,68,48.98,131.55,77.32,1.83
208,pachino,0,IT,1596062179,56,36.72,15.09,79.00,4.70
333,bafq,0,IR,1596062207,20,31.61,55.41,78.53,3.69


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = pd.DataFrame(ideal_df)
hotel_df['Hotel Name'] = ''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
City_ID,,,,,,,,,,
14,lodja,0,CD,1596062099,42,-3.48,23.43,73.13,0.43,
87,burriana,0,ES,1596062151,84,39.89,-0.09,78.01,3.53,
88,barcelos,0,PT,1596061919,93,41.54,-8.62,73.00,1.01,
111,lata,0,PT,1596062156,69,40.16,-8.33,73.00,2.68,
131,komsomolskiy,0,UZ,1596061980,44,40.43,71.72,75.20,4.70,
149,arles,0,FR,1596062165,65,43.68,4.63,78.80,2.24,
157,izvestkovyy,0,RU,1596062166,68,48.98,131.55,77.32,1.83,
208,pachino,0,IT,1596062179,56,36.72,15.09,79.00,4.70,
333,bafq,0,IR,1596062207,20,31.61,55.41,78.53,3.69,


In [13]:
target_search = 'hotel'
target_radius = 5000
target_type = 'lodging'
params = {"keyword": target_search,
        "radius": target_radius,
        'type': target_type,
        "key": g_key}

for index, row in hotel_df.iterrows():

    latitude = row['Lat']
    longitude = row['Lng']
    
    params['location'] = f'{latitude},{longitude}'

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_info = requests.get(base_url,params=params).json()
    
    try: 
        hotel_df.loc[index, 'Hotel Name'] = hotel_info['results'][0]['name']
    except:
        print(f"Problem finding hotels near: " + hotel_df.loc[index,'City'])    

Problem finding hotels near: lodja
Problem finding hotels near: lata
Problem finding hotels near: izvestkovyy
Problem finding hotels near: talakan
Problem finding hotels near: dawlatabad


In [14]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
City_ID,,,,,,,,,,
14,lodja,0,CD,1596062099,42,-3.48,23.43,73.13,0.43,
87,burriana,0,ES,1596062151,84,39.89,-0.09,78.01,3.53,Hotel Aloha Arenal Beach
88,barcelos,0,PT,1596061919,93,41.54,-8.62,73.00,1.01,Hotel Bagoeira
111,lata,0,PT,1596062156,69,40.16,-8.33,73.00,2.68,
131,komsomolskiy,0,UZ,1596061980,44,40.43,71.72,75.20,4.70,Hotel Mehmon Saroy
149,arles,0,FR,1596062165,65,43.68,4.63,78.80,2.24,Hôtel & Spa Jules César Arles - MGallery
157,izvestkovyy,0,RU,1596062166,68,48.98,131.55,77.32,1.83,
208,pachino,0,IT,1596062179,56,36.72,15.09,79.00,4.70,Hotel Villa Giulia
333,bafq,0,IR,1596062207,20,31.61,55.41,78.53,3.69,Alamdar Hotel


In [15]:
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
City_ID,,,,,,,,,,
87,burriana,0,ES,1596062151,84,39.89,-0.09,78.01,3.53,Hotel Aloha Arenal Beach
88,barcelos,0,PT,1596061919,93,41.54,-8.62,73.00,1.01,Hotel Bagoeira
131,komsomolskiy,0,UZ,1596061980,44,40.43,71.72,75.20,4.70,Hotel Mehmon Saroy
149,arles,0,FR,1596062165,65,43.68,4.63,78.80,2.24,Hôtel & Spa Jules César Arles - MGallery
208,pachino,0,IT,1596062179,56,36.72,15.09,79.00,4.70,Hotel Villa Giulia
333,bafq,0,IR,1596062207,20,31.61,55.41,78.53,3.69,Alamdar Hotel
353,muros,0,ES,1596062211,73,42.78,-9.06,73.99,1.01,A Casa da Fonte
372,florence,0,IT,1596062201,64,43.77,11.25,79.00,2.24,Four Seasons Hotel Firenze
379,belgrade,0,RS,1596061932,50,44.80,20.47,78.80,3.36,Square Nine Hotel Belgrade


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat","Lng"]]

In [17]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))